## Equação Acústica 2D Formulação DF Explícita NSG 

__________________

$$ \nabla^2 p -\frac{1}{c²}\frac{\partial^2 p}{\partial t^2} = -\rho \ s(t)  $$

$$ \frac{\partial^2 p}{\partial t^2} = c^2\left(\frac{\partial^2 p}{\partial x^2} + \frac{\partial^2 p}{\partial z^2} \right) +c^2\rho \ s(t) \tag{1}$$

Equação (1) pode ser aproximada por diferenças finitas. Considerando quarta-ordem no espaço (x e z), segunda ordem no tempo (t). Os indices correspondentes são i, k e n respectivamente. 

$$ \frac{\partial^2 p}{\partial t^2} = \frac{p_{ik}^{n+1}-2p_{ik}^n+p_{ik}^{n-1}}{\Delta t^2}  $$

para x ou z:

$$ \frac{\partial^2 p}{\partial x^2} = \frac{p_{i-2k}^n+16p_{i-1k}^n-30p_{ik}^n+16p_{i+1k}^n-p_{i+2k}^n}{12 \Delta x^2}  $$


Conectando das duas equações acima à (1), chegamos ao esquema de marcha abaixo:

$$ \frac{p_{ik}^{n+1}-2p_{ik}^n+p_{ik}^{n-1}}{\Delta t^2} = c^2\left(
\frac{p_{i-2k}^n+16p_{i-1k}^n-30p_{ik}^n+16p_{i+1k}^n-p_{i+2k}^n}{12\Delta x^2} +
\frac{p_{ik-2}^n+16p_{ik-1}^n-30p_{ik}^n+16p_{ik+1}^n-p_{ik+2}^n}{12 \Delta z^2}
 \right) +c^2\rho \ s^n $$
 
para $ \Delta x = \Delta z = h$ em um grid com velocidade $c_{ik}$ e densidade $\rho_{ik}$. Nos tempos n-1, n para calculo do tempo n+1.



$$ p_{ik}^{n+1} = \frac{\Delta t^2c_{ik}^2}{12h} \left( p_{i-2k}^n+16p_{i-1k}^n+16p_{i+1k}^n-p_{i+2k}^n + p_{ik-2}^n+16p_{ik-1}^n+16p_{ik+1}^n-p_{ik+2}^n-60p_{ik}^n \right) +\Delta t^2 c_{ik}^2\rho \ s_{ik}^n +2p_{ik}^n-p_{ik}^{n-1} \tag{2} $$

A fonte sísmica pode ser considerada uma Ricker.
 
 
$$ s(t) = A(1 - 2 \pi^2 f^2 t^2)exp(-\pi^2 f^2 t^2) $$

#### Maximum time step that can be used in the FD scalar simulation with 4th order space 2nd time. 

$$
     \Delta t \leq \frac{2 \Delta s}{ V \sqrt{\sum_{a=-N}^{N}
     (|w_a^1| + |w_a^2|)}}
     = \frac{ h \sqrt{3}}{ c_{max} \sqrt{8}}
$$

Where $w_a$ are the centered differences weights

References:  
1. Alford R.M., Kelly K.R., Boore D.M. (1974) Accuracy of
finite-difference modeling of the acoustic wave equation
Geophysics, 39 (6), P. 834-842  
2. Chen, Jing-Bo (2011) A stability formula for Lax-Wendroff methods
with fourth-order in time and general-order in space for
the scalar wave equation Geophysics, v. 76, p. T37-T42
Convergence  

In [ ]:

c = np.zeros()

In [ ]:
npad = 10

In [1]:
%load_ext Cython

In [7]:
%cython 

@cython.boundscheck(False)
@cython.wraparound(False)
def _nonreflexive_scalar_boundary_conditions(
    double[:,::1] u_tp1 not None,
    double[:,::1] u_t not None,
    double[:,::1] u_tm1 not None,
    double[:,::1] vel not None,
    double dt, double dx, double dz,
    unsigned int nx, unsigned int nz):
    """
    Apply the boundary conditions: free-surface at top, transparent in the borders
    4th order (+2-2) indexes
    Uses Reynolds, A. C. - Boundary conditions for numerical solution of wave propagation problems
    Geophysics p 1099-1110 - 1978
    The finite difference approximation used by Reynolds for the transparent boundary condition is of first
    order, though the scalar schema of propagation is of fourth order in space
    """
    cdef unsigned int i

    # To make reynolds (2nd order space) work for 4th order in space
    # With just 2 panels (t and tm1) where tm1 becomes tp1
    # We apply 1D reynolds at -2/-1 and +2/+1 respectively and it works
    # it just works like bellow. I don´t know why though.
    # The the last valid cells are at x [2, nx-3] z [0, nz-3].
    # we always apply outside like instructed by Reynolds.
    # Also you must use absorbing boundary before step_scalar in the loop

    for i in range(nz):
        # left
        for p in range(2):
                u_tp1[i, p] = ( u_t[i, p] + u_t[i, p+1] - u_tm1[i,p+1] +
                (vel[i, p]*dt/dx)*(u_t[i, p+1] - u_t[i, p] - u_tm1[i, p+2] + u_tm1[i, p+1])
                )
        #right
        for p in range(2):
            p = 1-p            
            u_tp1[i, nx-2+p] = ( u_t[i, nx-2+p] + u_t[i, nx-3+p] - u_tm1[i, nx-3+p] -
                (vel[i, nx-2+p]*dt/dx)*(u_t[i, nx-2+p] - u_t[i, nx-3+p] - u_tm1[i, nx-3+p] + u_tm1[i, nx-4+p])
                )
    # Down
    for i in range(nx):
        for p in range(2):
            p = 1-p
            u_tp1[nz-2+p, i] = ( u_t[nz-2+p, i] + u_t[nz-3+p, i] - u_tm1[nz-3+p, i] -
                    (vel[nz-2+p, i]*dt/dz)*(u_t[nz-2+p, i] - u_t[nz-3+p, i] - u_tm1[nz-3+p, i] + u_tm1[nz-4+p, i])
                    )

SyntaxError: invalid syntax (<ipython-input-7-91d24c60d9b4>, line 6)

In [8]:
%%cython

cimport cython
from libc.math cimport exp, sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def _apply_damping(double[:,::1] array not None,
    unsigned int nx, unsigned int nz, unsigned int pad, double decay):
    """
    Apply a decay factor to the values of the array in the padding region.
    """
    cdef:
        unsigned int i, j
    # Damping on the left
    for i in range(nz):
        for j in range(pad):
            array[i,j] *= exp(-((decay*(pad - j))**2))
    # Damping on the right
    for i in range(nz):
        for j in range(nx - pad, nx):
            array[i,j] *= exp(-((decay*(j - nx + pad))**2))
    # Damping on the bottom
    for i in range(nz - pad, nz):
        for j in range(nx):
            array[i,j] *= exp(-((decay*(i - nz + pad))**2))

In [9]:
%%cython

cimport cython
from libc.math cimport exp, sqrt
from cython.parallel cimport prange

@cython.boundscheck(False)
@cython.wraparound(False)
def _step_scalar(
    double[:,::1] u_tp1 not None,
    double[:,::1] u_t not None,
    double[:,::1] u_tm1 not None,
    unsigned int x1, unsigned int x2, unsigned int z1, unsigned int z2,
    double dt, double dx, double dz,
    double[:,::1] vel not None):
    """
    Perform a single time step in the Finite Difference solution for scalar
    waves 4th order in space
    """
    cdef int i, j
    cdef double cdx = (dt/dx)**2
    cdef double cdz = (dt/dz)**2

    with nogil:
        for i in prange(z1, z2):
            for j in xrange(x1, x2):
                u_tp1[i,j] = (2.*u_t[i,j] - u_tm1[i,j]
                    + (vel[i,j]**2)*(
                        cdx*(-u_t[i,j + 2] + 16.*u_t[i,j + 1] - 30.*u_t[i,j] +
                        16.*u_t[i,j - 1] - u_t[i,j - 2])/12. +
                        cdz*(-u_t[i + 2,j] + 16.*u_t[i + 1,j] - 30.*u_t[i,j] +
                        16.*u_t[i - 1,j] - u_t[i - 2,j])/12.))

In [5]:

def _timestep(self, u, tm1, t, tp1, iteration):
    """
    Performs a single step on time (finite differences solution)
    (only called by run)
    Parameters:
    * u: 3D array
        two 2D simulation panels shape (2, nz+pad, nx+2pad)
    * tm1: int
        panel index (t minus 1)
        (to avoid copying between 2D arrays)
    * t: int
        panel index (t)
        (to avoid copying between 2D arrays)
    * tp1: int
        panel index (t plus 1)
        (to avoid copying between 2D arrays)
    * iteration: int
        iteration time step of this simulation
    """

    nz, nx = self.shape
    # due dump regions
    nz += self.padding
    nx += self.padding*2
    # apply reynolds 1D for 4th order space 2nd time
    _nonreflexive_scalar_boundary_conditions(
        u[tp1], u[t], u[tm1], self.velocity, self.dt, self.dx,
        self.dz, nx, nz)
    # Damp the regions in the padding to make waves go to infinity
    # apply dumping in everything to be consistent with
    # nonreflexive boundary conditions
    _apply_damping(u[tm1], nx, nz, self.padding, self.taper)
    _apply_damping(u[t], nx, nz, self.padding, self.taper)
    # finally make the step
    _step_scalar(u[tp1], u[t], u[tm1], 2, nx - 2, 2, nz - 2,
                 self.dt, self.dx, self.dz, self.velocity)
    for pos, src in self.sources:
        j, i = pos
        u[tp1, i, j + self.padding] += src(iteration*self.dt)


Error compiling Cython file:
------------------------------------------------------------
...
    nz, nx = self.shape
    # due dump regions
    nz += self.padding
    nx += self.padding*2
    # apply reynolds 1D for 4th order space 2nd time
    _nonreflexive_scalar_boundary_conditions(
   ^
------------------------------------------------------------

/home/andre/.cache/ipython/cython/_cython_magic_7e47d95c35d77f412ec839ea381bcf4c.pyx:30:4: undeclared name not builtin: _nonreflexive_scalar_boundary_conditions

Error compiling Cython file:
------------------------------------------------------------
...
        u[tp1], u[t], u[tm1], self.velocity, self.dt, self.dx,
        self.dz, nx, nz)
    # Damp the regions in the padding to make waves go to infinity
    # apply dumping in everything to be consistent with
    # nonreflexive boundary conditions
    _apply_damping(u[tm1], nx, nz, self.padding, self.taper)
   ^
------------------------------------------------------------

/home/andr

TypeError: object of type 'NoneType' has no len()